<a href="https://colab.research.google.com/github/Lin-Sinorodin/Military_Vehicles_Tracking/blob/main/Military_Vehicles_Tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Military Vehicles Tracking

> More info can be found on the [GitHub](https://github.com/Lin-Sinorodin/Military_Vehicles_Tracking) page of the project

In [ ]:
#@title Downloads { vertical-output: true, display-mode: "form" }
#@markdown __Run this cell once__

# clone the project from GitHub
!git clone https://github.com/Lin-Sinorodin/Military_Vehicles_Tracking
%cd Military_Vehicles_Tracking/

# download the trained model's weights
!wget -qq https://github.com/Lin-Sinorodin/Military_Vehicles_Tracking/releases/download/v1.0.0/one_class_YOLOV5s.pt -P weights/
!wget -qq https://github.com/Lin-Sinorodin/Military_Vehicles_Tracking/releases/download/v1.0.0/one_class_YOLOv5x.pt -P weights/
!wget -qq https://github.com/Lin-Sinorodin/Military_Vehicles_Tracking/releases/download/v1.0.0/person_military_seperate_YOLOv5l.pt -P weights/

# download test videos
!wget -qq https://github.com/Lin-Sinorodin/Military_Vehicles_Tracking/releases/download/v1.0.0/test1.mp4 -P test_videos/
!wget -qq https://github.com/Lin-Sinorodin/Military_Vehicles_Tracking/releases/download/v1.0.0/test3.mp4 -P test_videos/
!wget -qq https://github.com/Lin-Sinorodin/Military_Vehicles_Tracking/releases/download/v1.0.0/test4.mp4 -P test_videos/
!wget -qq https://github.com/Lin-Sinorodin/Military_Vehicles_Tracking/releases/download/v1.0.0/test5.mp4 -P test_videos/
!wget -qq https://github.com/Lin-Sinorodin/Military_Vehicles_Tracking/releases/download/v1.0.0/test10.mp4 -P test_videos/
!wget -qq https://github.com/Lin-Sinorodin/Military_Vehicles_Tracking/releases/download/v1.0.0/test13.mp4 -P test_videos/
!wget -qq https://github.com/Lin-Sinorodin/Military_Vehicles_Tracking/releases/download/v1.0.0/test16.mp4 -P test_videos/
!wget -qq https://github.com/Lin-Sinorodin/Military_Vehicles_Tracking/releases/download/v1.0.0/test17.mp4 -P test_videos/

In [28]:
#@title Setup { display-mode: "form" }
#@markdown __Run this cell once__

import io
import os
import torch
from pprint import pprint
from base64 import b64encode
import matplotlib.pyplot as plt
from IPython.display import HTML, clear_output

from Military_Vehicles_Tracking.src.video import Video
from Military_Vehicles_Tracking.src.tracker import MultiObjectTracker
from Military_Vehicles_Tracking.src.plot_utils import plot_bounding_boxes

!pip install -qq --ignore-installed PyYAML

# setup GPU device if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Runtime details:')
print(f'torch {torch.__version__}')
print(torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU')

# get available weights
weights_path = 'weights'
weights_names = os.listdir(weights_path)
weights = {name: f'{weights_path}/{name}' for name in weights_names}

# get available test videos
test_videos_path = 'test_videos'
for i, video_name in enumerate(sorted(os.listdir(test_videos_path))):
    os.rename(f'{test_videos_path}/{video_name}', f'{test_videos_path}/test_{i+1}.mp4')
for i, video_name in enumerate(sorted(os.listdir(test_videos_path))):
    os.rename(f'{test_videos_path}/{video_name}', f'{test_videos_path}/test{i+1}.mp4')
videos_names = sorted(os.listdir(test_videos_path))
videos_paths = [f'{test_videos_path}/{name}' for name in videos_names]

# create folders for the results
for weights_name, weights_path in weights.items():
    os.makedirs(f'results/{weights_name}', exist_ok=True)

print('\nAvailable trained weights:')
pprint(list(weights.values()))

print('\nAvailable test videos:')
pprint(videos_paths)

Runtime details:
torch 1.10.0+cu111
Tesla K80

Available trained weights:
['weights/one_class_YOLOV5s.pt',
 'weights/one_class_YOLOv5x.pt',
 'weights/person_military_seperate_YOLOv5l.pt.1',
 'weights/person_military_seperate_YOLOv5l.pt',
 'weights/one_class_YOLOv5x.pt.1',
 'weights/one_class_YOLOV5s.pt.1']

Available test videos:
['test_videos/test1.mp4',
 'test_videos/test10.mp4',
 'test_videos/test11.mp4',
 'test_videos/test12.mp4',
 'test_videos/test13.mp4',
 'test_videos/test14.mp4',
 'test_videos/test15.mp4',
 'test_videos/test16.mp4',
 'test_videos/test2.mp4',
 'test_videos/test3.mp4',
 'test_videos/test4.mp4',
 'test_videos/test5.mp4',
 'test_videos/test6.mp4',
 'test_videos/test7.mp4',
 'test_videos/test8.mp4',
 'test_videos/test9.mp4']


In [ ]:
#@title Initialize the Object Detectors { display-mode: "form" }
#@markdown __Run this cell once__

detectors = [
    torch.hub.load('ultralytics/yolov5', 'custom', weights_path).to(device) 
    for weights_path in weights.values()
]

## Multiple Object Tracking

In [29]:
#@title Choose Test Video and Detector
video = "test2.mp4" #@param ["test1.mp4", "test2.mp4", "test3.mp4", "test4.mp4", "test5.mp4", "test6.mp4", "test7.mp4", "test8.mp4"]
detector = 'One class YOLOv5s' #@param ["One class YOLOv5s", "One class YOLOv5x", "Multiple class YOLOv5l"]

video_name = video
detector_map = {"One class YOLOv5s": 0,
                "One class YOLOv5x": 1,
                "Multiple class YOLOv5l": 2}
detector_idx = detector_map[detector]

# initialize a video
video = Video(f'{test_videos_path}/{video_name}')

# initialize object detector
weights_name = list(weights.keys())[detector_idx]
detector = detectors[detector_idx]

# initialize object tracker
results_path = f'results/{weights_name}/{video_name}'
tracker = MultiObjectTracker(video, results_path, detector)

# iterate over the frames in the video
for frame, bounding_boxes in tracker:
    plot_bounding_boxes(frame, bounding_boxes)
    tracker.video_writer.write(frame)

# save the results
tracker.video_writer.release()
print(f'\n\n=> ({weights_name}) - Saved results for: {video_name}')
print(f'   result saved in {results_path}')

# convert video for display
print('\nConverting video to displayable format...')
path_webm = results_path.replace('mp4', 'webm')
!ffmpeg -loglevel warning -y -i "$results_path" -vcodec vp9 "$path_webm"

# load video for display
with  io.open(path_webm,'r+b') as f:
    vid = f.read()

# display video
data_url = "data:video/webm;base64," + b64encode(vid).decode()
HTML("""
<video width=800 controls>
    <source src="%s" type="video/webm">
</video>
""" % data_url)

Loading weights from src/deep_sort_pytorch/deep_sort/deep/checkpoint/ckpt.t7... Done!
Tracking:  98%|█████████▊| 454/462 [00:42<00:00, 10.56frame/s]




=> (one_class_YOLOV5s.pt) - Saved results for: test2.mp4
   result saved in results/one_class_YOLOV5s.pt/test2.mp4

Converting video to displayable format...
